# FeatureSelection

In [57]:

#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [58]:
df = pd.read_csv("..\..\Project_DATA\HealthCarePred_flat_1.csv")

### Univariate Analysis: Corellation of outcome variable with all other variables:

In [88]:
featuresVotingList = []

rows, cols = df.shape
features = list(df.columns)
for i in range(cols - 1):
    curr_corr = df["Spending2012"].corr(df[features[i]])
    currFeature = {'Feature': features[i], 'Univariable': 1 if curr_corr > 0.2 else 0 }
    featuresVotingList.append(currFeature)    
featuresVoting = pd.DataFrame(featuresVotingList)        

In [89]:
featuresVoting.head()

#featuresVoting[featuresVoting.Univariable > 0.2].sort_values(by='Univariable', ascending=False)

,Feature,Univariable
0,State_County_CD,0
1,AGE,0
2,SEX,0
3,race,0
4,CCS101,1
5,CCS104,0
6,CCS105,0
7,CCS106,0
8,CCS108,1
9,CCS257,1


### Multivariate analysis: 


In [90]:
X = df.drop("Spending2012",1)   #Feature Matrix
y = df["Spending2012"]          #Target Variable

#### Lasso

In [91]:
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

Best alpha using built-in LassoCV: 148.398375
Best score using built-in LassoCV: 0.428442


In [100]:
featuresVoting['Lasso1'] = coef.values
featuresVoting.loc[featuresVoting['Lasso1'] != 0, 'Lasso'] = 1
featuresVoting.loc[featuresVoting['Lasso1'] == 0, 'Lasso'] = 0
featuresVoting.Lasso = pd.to_numeric(featuresVoting.Lasso, downcast='integer')
featuresVoting = featuresVoting.drop(['Lasso1'], axis=1)
featuresVoting.head()

,Feature,Univariable,Lasso
0,State_County_CD,0,1
1,AGE,0,1
2,SEX,0,0
3,race,0,0
4,CCS101,1,1


In [101]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

Lasso picked 173 variables and eliminated the other 286 variables


#### Recursive Feature Elimination - RFE

In [104]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR

estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)

KeyboardInterrupt: 

In [ ]:
rfe_sup = selector.support_

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassfier
from sklearn.feature_selection import SelectFromModel

sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X, y)


In [ ]:
rf_sup = sel.get_support()

#### Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

XGBC = XGBClassifier()
XGBC.fit(X, y)
# feature importance


In [ ]:
gb_imp = XGBC.feature_importances_

#### SVM